## 正则化、dropout、步长调整

In [161]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [162]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [163]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  hidden_node = 100 #这个一大就崩
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_node]))
  biases1 = tf.Variable(tf.zeros([hidden_node]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_node, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  y1 = tf.matmul(tf_train_dataset, weights1) + biases1
  h1_output = tf.nn.relu(y1)
  train_predict = tf.matmul(h1_output, weights2) + biases2

  v1 = tf.matmul(tf_valid_dataset, weights1) + biases1
  v1_output = tf.nn.relu(v1)
  valid_predict = tf.matmul(v1_output, weights2) + biases2

  t1 = tf.matmul(tf_test_dataset, weights1) + biases1
  t1_output = tf.nn.relu(t1)
  test_predict = tf.matmul(t1_output, weights2) + biases2

  #正则化!!!!!amazing!!!加了这个之后准确率提高了！！！
  l2_loss = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2)
  beta = 0.02
    
  #训练时dropout
  keep_prob = tf.placeholder(tf.float32)
  h1_drop = tf.nn.dropout(h1_output,keep_prob)
  training_predict = tf.matmul(h1_drop, weights2) + biases2
    
  loss = beta*l2_loss + tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=training_predict))
  
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  #Learning Rate Decay自动调整步长
  cur_step = tf.Variable(0)
  start_rate = 0.1
  learning_rate = tf.train.exponential_decay(start_rate, cur_step, 10000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=cur_step)
    
  train_prediction = tf.nn.softmax(train_predict)
  valid_prediction = tf.nn.softmax(valid_predict)
  test_prediction = tf.nn.softmax(test_predict)

In [165]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [166]:
num_steps = 1001
#网络节点隐藏层太多，一跑就崩
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob:0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 773.644043
Minibatch accuracy: 12.5%
Validation accuracy: 13.4%
Minibatch loss at step 500: 82.073059
Minibatch accuracy: 68.8%
Validation accuracy: 73.4%
Minibatch loss at step 1000: 11.833797
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Test accuracy: 86.9%
